In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=False)   #link to google drive

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/My Drive/perception projects/object tracking")  #change directory to directory with needed files for YOLO tracking
!ls      #print out files in directory

images	movie.mp4  YOLO_simple.ipynb  yolo_Starter.ipynb  yolov3


In [ ]:
!python3 -m pip install yolov4==2.0.2 # After Checking, YOLO 2.0.2 works without modifying anything. Otherwise keep 1.2.1from yolov4.tf import YOLOv4
from yolov4.tf import YOLOv4
import tensorflow as tf
import time


In [ ]:
yolo = YOLOv4(tiny=False)
yolo.classes = "Yolov4/coco.names"
yolo.make_model()
yolo.load_weights("Yolov4/yolov4.weights", weights_type="yolo")

def run_obstacle_detection(img):
    start_time=time.time()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_image = yolo.resize_image(img)

    # 0 ~ 255 to 0.0 ~ 1.0
    resized_image = resized_image / 255.      #normalize image

    #input_data == Dim(1, input_size, input_size, channels)
    input_data = resized_image[np.newaxis, ...].astype(np.float32)  #add one more dimension to image structure and change data type
    candidates = yolo.model.predict(input_data)   #model predict

    _candidates = []
    result = img.copy()     #create a copy of the image
    for candidate in candidates:
        batch_size = candidate.shape[0]
        grid_size = candidate.shape[1]
        _candidates.append(tf.reshape(candidate, shape=(1, grid_size * grid_size * 3, -1)))   #reshape candidate

        #candidates == Dim(batch, candidates, (bbox))
        candidates = np.concatenate(_candidates, axis=1)
        
        #pred_bboxes == Dim(candidates, (x, y, w, h, class_id, prob))
        pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0], iou_threshold=0.35, score_threshold=0.40)
        pred_bboxes = pred_bboxes[~(pred_bboxes==0).all(1)] #https://stackoverflow.com/questions/35673095/python-how-to-eliminate-all-the-zero-rows-from-a-matrix-in-numpy?lq=1
        pred_bboxes = yolo.fit_pred_bboxes_to_original(pred_bboxes, img.shape)
        exec_time = time.time() - start_time
        #print("time: {:.2f} ms".format(exec_time * 1000))
        result = yolo.draw_bboxes(img, pred_bboxes)
        result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    return result, pred_bboxes


In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimp
import numpy as npc 
!pwd
path = "images/man_bike.jpg"
image = mpimg.imread(path)
plt.imshow(image)

result, pred_bboxes = run_obstacle_detection(image)
fig_camera = plt.figure(figsize=(20, 20))
ax_lidar = fig_camera.subplots()
ax_lidar.imshow(result)
plt.show()




Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from moviepy.editor import VideoFileClip
video_file = "images/day_drive_2mins.mp4"
clip = VideoFileClip(video_file).subclip(0,50)
white_clip = clip.fl_image(run_obstacle_detection)
%time white_clip.write_videofile("output_daydrive.mp4",audio=False)